In [1]:
import os
os.getcwd()

'/teamspace/studios/this_studio'

In [2]:
import re
import json
import string
# from transformers import Wav2Vec2CTokenizer
from datasets import load_from_disk, load_dataset

chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\״\%\‘\”\�\'\]\[\{\}\־\u202b]'
hebrew_letters = set("אבגדהוזחטיכךלמםנןסעפףצץקרשת")

dataset= load_dataset("ivrit-ai/whisper-training")


In [10]:

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["text"]).lower()
    return batch

def filter_hebrew_samples(dataset):
    def contains_only_hebrew(text):
        return all(char in hebrew_letters or char.isspace() for char in text)
    filtered_dataset = dataset.filter(lambda example: contains_only_hebrew(example["transcription"]))

    original_samples = dataset['transcription']
    filtered_samples = [sample for sample in original_samples if contains_only_hebrew(sample)]
    
    filtered_out_indices = [i for i, sample in enumerate(original_samples) if not contains_only_hebrew(sample)]
    
    return filtered_dataset, filtered_out_indices

def get_vocab(dataset):
    vocab = {}
    for example in dataset:
        for char in example["text"]:
            if char in hebrew_letters or char == '|':
                if char not in vocab:
                    vocab[char] = len(vocab)
    return vocab
# Example datasetz
# Preprocess dataset
# dataset["train"] = filter_hebrew_samples(dataset["train"])

In [4]:
samples= dataset["train"].select(range(1000))
samples= samples.map(remove_special_characters)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
filtered_samples, filtered_out_indices= filter_hebrew_samples(samples)


In [6]:
len(filtered_samples)

859

In [26]:
for index in filtered_out_indices:
    print(f"Filtered out sample {index}: {samples['transcription'][index]}")

# Find and print any character in the sample text that is not in the hebrew_letters set
    missing_chars = {char for char in filtered_samples[index] if char not in hebrew_letters and not char.isspace()}

    print("Missing characters: ", missing_chars)

# print("\nFiltered samples:")
# for sample in filtered_samples:
#     print(sample)

Filtered out sample 10: בעצם אני פאונדר ceo
Missing characters:  set()
Filtered out sample 11: זה לא יכול להיות מ0 ל1 התקוות האלה
Missing characters:  set()
Filtered out sample 30: יש פה בארץ יש אין ספור דוגמאות אתה מזמין מונית באובר זה  search כל החיפושים האלה קורים כל הזמן
Missing characters:  set()
Filtered out sample 32: אז תקשיבי stranger things את חייבת לראות את זה אנחנו עדיין לא סגורים על זה בבית
Missing characters:  set()
Filtered out sample 34: i became friends with that guy in kenya
Missing characters:  set()
Filtered out sample 35: oh man he knows the lyrics together so why did he do it together
ומכניס את הבוק בוק

Missing characters:  set()
Filtered out sample 36: מוצר שלך ככה וככה או החברה שלך עושה ככה וככה ואתה שואל אוקיי איזה kpi תשפר לי וkpi יכול להיות הכנסות
Missing characters:  set()
Filtered out sample 39: שאוהב את אלה מהרמב״ם והרמב״ם חלק מהדברים מסעדיה גאון כאילו תראו איך היהדות
Missing characters:  set()
Filtered out sample 43: מה היה הצילום שעליו קיבלת את המאמר בn

Filtered out sample 574: בעצם מה מה מה שכחלון עשה ב2011 זה בעצם הוא הכריח את החברות הקיימות
Missing characters:  set()
Filtered out sample 606: זה התחיל בזה שזה מדבר על סוגיות חברתיות אבל בהומור ובכיף ואז אני כמו סטאוואז הם בונים גם מיתולוגיה תבין זה לא רק מה הוא עושה זאת יודעת על משהו שקרה לו לפני 200 שנה אתה לא ראית את זה אף פעם בטלוויזיה מדמיין את המיתולוגיה ואז יש לך מלא דמויות מסוגים שונים והווילנס חוזרים
Missing characters:  set()
Filtered out sample 629: זה נהיה במיוחד פופולארי אחרי פרק הסיום של game of thrones והאורח שלנו היום הוא לא אחר מאשר דרור גלוברמן
Missing characters:  set()
Filtered out sample 632: במאה ומשהו שנים של מקרו כלכלה ושל מדיניות מוניטרית ושל כלים שהצליחו ושל כאלה שהצליחו פחות אבל אם אתה ליברטריאן אנרכיסט ואם אתה חושב שכל המערכת מושכת אז אתה בטח לא רוצה להתבסס על הידע שלה זורק הובס וזהו אתה לא צריך באמת להתעמק אתה יכול להגיד כן עדיף אפילו לוק לוק הובס כן מספיק לזרוק שמות של פילוסופים אנגלים מהמאה ה16 מהמאה ה18 הכל סבבה נכון נכון אז אנחנו התחלנו הנקודה הראשונה 

IndexError: list index out of range

In [7]:
import transformers
from transformers import Wav2Vec2CTCTokenizer

In [14]:
print("Getting Vocab")
vocab = get_vocab(filtered_samples)
vocab["|"] = vocab.get("|", len(vocab))
vocab["[UNK]"] = len(vocab)
vocab["[PAD]"] = len(vocab)
vocab["<s>"] = len(vocab)
vocab["</s>"] = len(vocab)
print("printing vocab with length: ", len(vocab))
with open(f"vocab.json", "w") as f:
    json.dump(vocab, f)

Getting Vocab


printing vocab with length:  32


In [ ]:
tokenizer = Wav2Vec2CTokenizer.from_pretrained(finetuned_model_path,
                                               unk_token="[UNK]",
                                               pad_token="[PAD]",
                                               word_delimiter_token="|")

In [ ]:
feature_extractor = MODEL_CONFIG['feature_extractor'].from_pretrained(MODEL_CONFIG['model_name'])
processor = MODEL_CONFIG['processor'](feature_extractor=feature_extractor, tokenizer=tokenizer)
print("saving processor")
processor.save_pretrained(local_model_path + '-finetuned')